In [25]:
import wikipediaapi
from langchain import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Qdrant
from langchain.llms import Ollama
from langchain.chains import RetrievalQA

In [11]:
wiki_wiki = wikipediaapi.Wikipedia(
    user_agent='Giskard application',
    language='en',
    extract_format=wikipediaapi.ExtractFormat.WIKI
)
p_wiki = wiki_wiki.page('Virat_Kohli')
data = p_wiki.text

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100, add_start_index=True)

In [14]:
docs = text_splitter.create_documents([data])

In [19]:
embeddings = OllamaEmbeddings(
    model='nomic-embed-text'
)

In [23]:
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="wiki_data",
)

In [24]:
retriever = qdrant.as_retriever()

In [28]:
PROMPT_TEMPLATE = """You are the Wikipedia Assitant.
Your task is to answer common questions on Virat Kohli.
You will be given a question and relevant excerpts from the wikipedia page.
Please provide short and clear answers based on the provided context. Be polite and helpful.

Context:
{context}

Question:
{question}

Your answer:
"""

llm = Ollama(model="phi3", temperature=0)
prompt = PromptTemplate(template=PROMPT_TEMPLATE, input_variables=["question", "context"])
qa_chain = RetrievalQA.from_llm(llm=llm, retriever=retriever, prompt=prompt)

# Test that everything works
qa_chain.invoke({"query": "What is his wife's name?"})

{'query': "What is his wife's name?",
 'result': "Virat Kohli's wife's name is Anushka Sharma. They got married on 11 December 2017 in Florence, Italy. Their daughter's name is Vamika and their son's name is Akaay."}

In [29]:
import giskard
import pandas as pd


def model_predict(df: pd.DataFrame):
    """Wraps the LLM call in a simple Python function.

    The function takes a pandas.DataFrame containing the input variables needed
    by your model, and must return a list of the outputs (one for each row).
    """
    return [qa_chain.run({"query": question}) for question in df["question"]]


# Don’t forget to fill the `name` and `description`: they are used by Giskard
# to generate domain-specific tests.
giskard_model = giskard.Model(
    model=model_predict,
    model_type="text_generation",
    name="Wikipedia Question Answering",
    description="This model answers any question about a person's wikipedia page",
    feature_names=["question"],
)

2024-05-28 22:31:44,112 pid:10604 MainThread giskard.models.automodel INFO     Your 'prediction_function' is successfully wrapped by Giskard's 'PredictionFunctionModel' wrapper class.


In [30]:
# Optional: let’s test that the wrapped model works
examples = [
    "When did he hit his first test century?",
    "How much runs did he score in 2016 IPL?",
]
giskard_dataset = giskard.Dataset(pd.DataFrame({"question": examples}), target=None)

print(giskard_model.predict(giskard_dataset).prediction)

2024-05-28 22:31:51,476 pid:10604 MainThread giskard.datasets.base INFO     Your 'pandas.DataFrame' is successfully wrapped by Giskard's 'Dataset' wrapper class.
2024-05-28 22:31:51,493 pid:10604 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'question': 'object'} to {'question': 'object'}
2024-05-28 22:33:27,081 pid:10604 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (2, 1) executed in 0:01:35.608124
["Virat Kohli scored his first Test century on 23 November 2 Written by a language model AI. Please note that while I strive for accuracy, it's always good to verify important information from official sources.\n\nVirat Kohli hit his first Test century on 23rd November 2014 against the West Indies in Mumbai."
 'In the 2016 Indian Premier League (IPL), Virat Kohli scored a total of 973 runs.']


In [31]:
report = giskard.scan(giskard_model, giskard_dataset, only="hallucination")

🔎 Running scan…
Estimated calls to your model: ~30
Estimated LLM calls for evaluation: 22

2024-05-28 22:34:02,411 pid:10604 MainThread giskard.scanner.logger INFO     Running detectors: ['LLMImplausibleOutputDetector', 'LLMBasicSycophancyDetector']
Running detector LLMImplausibleOutputDetector…
2024-05-28 22:34:18,622 pid:10604 MainThread httpx        INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-28 22:34:18,631 pid:10604 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'question': 'object'} to {'question': 'object'}
2024-05-28 22:42:59,175 pid:10604 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (10, 1) executed in 0:08:40.548535
2024-05-28 22:43:02,469 pid:10604 MainThread httpx        INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-28 22:43:04,819 pid:10604 MainThread httpx        INFO     HTTP Request: POST https://api.openai.co

In [ ]:
display(report)

In [ ]:
full_report = giskard.scan(giskard_model, giskard_dataset)

In [ ]:
display(full_report)

# Save it to a file
full_report.to_html("scan_report.html")